### Import needed libraries 

In [36]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity, sigmoid_kernel

### Loading data

In [28]:
data = pd.read_csv('data.csv')
data = data.iloc[:100, :]
data

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.2110,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.6650,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.3410,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.1600,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.1660,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.1010,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.3090,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.3810,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.1930,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.2290,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.1730,1921,0.992,['Morton Downey'],0.380,170240,0.1170,0,3tEwdlwf8ZrxpksJorI5Tc,0.000001,7,0.1050,-13.298,1,That's How You Spell Ireland,0,1921,0.0415,86.493
96,0.2190,1921,0.672,['Mehmet Kemiksiz'],0.403,396356,0.3850,0,47LO5zrjx6ShMymLqp80ef,0.000339,11,0.2110,-6.507,0,Şühedâ Gövdesi Bir Baksana Dağlar Taşlar,0,1921,0.0315,123.839
97,0.9190,1921,0.959,['Morton Downey'],0.829,153827,0.3330,0,4BWXQSxvAa73YTLVLlMtGp,0.000362,5,0.0841,-11.895,1,It's the Same Old Shillelegh,0,1921,0.1030,132.606
98,0.4510,1921,0.995,['Ignacio Corsini'],0.592,168053,0.0322,0,4Rv29R4ydhHN9tYPfrZxTz,0.057300,5,0.2970,-25.844,0,China Hereje - Remasterizado,0,1921-03-20,0.2090,69.958


# Understanding the dataset

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   valence           100 non-null    float64
 1   year              100 non-null    int64  
 2   acousticness      100 non-null    float64
 3   artists           100 non-null    object 
 4   danceability      100 non-null    float64
 5   duration_ms       100 non-null    int64  
 6   energy            100 non-null    float64
 7   explicit          100 non-null    int64  
 8   id                100 non-null    object 
 9   instrumentalness  100 non-null    float64
 10  key               100 non-null    int64  
 11  liveness          100 non-null    float64
 12  loudness          100 non-null    float64
 13  mode              100 non-null    int64  
 14  name              100 non-null    object 
 15  popularity        100 non-null    int64  
 16  release_date      100 non-null    object 
 17

# Scaling the numerical features

In [21]:
feature_columns = ['acousticness', 'danceability', 'duration_ms', 'energy',
                  'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
                  'speechiness', 'tempo', 'valence',]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(data[feature_columns])

# Building Recommender System using Cosine Similarity

In [35]:
indeces = pd.Series(data.index, index = data['name']).drop_duplicates()
indeces

cosine = cosine_similarity(X_scaled)
cosine.shape

def recommender(song_name, model_type = cosine):
    index = indeces[song_name]
    score = list(enumerate(model_type[indeces[song_name]]))
    similarity_score = sorted(score, key = lambda x: x[1], reverse = True)[1:4]
    top_songs_index = [i[0] for i in similarity_score]    
    top_songs = data['name'].iloc[top_songs_index]
    
    return top_songs

print("Recommended Songs:")
print(recommender('Danny Boy',cosine).values)


Recommended Songs:
['Come Back To Erin' 'April Showers' 'When Irish Eyes Are Smiling']


In [37]:
sigmoid = sigmoid_kernel(X_scaled)


print("Recommended Songs:")
print(recommender('Danny Boy',sigmoid).values)


Recommended Songs:
['Il Etait Syndiqué' 'Gati Harjuna Mangsah' 'Rákóczy March']
